In [ ]:
# model_experiments.ipynb
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler




In [7]:
import pandas as pd

def load_data(file_path):
    # Read the CSV file
    data = pd.read_csv(file_path, header=None, skiprows=1)  # Skip the first row if it contains incorrect headers
    
    # Assign column names based on the structure of the file
    data.columns = [
        "Hue", "Value", "Chroma", "r", "g", "b", 
        "L", "A", "B"
    ]
    
    # Inspect the data
    print(data.head())
    print(data.info())
    
    # Check for non-numeric values in the dataset
    non_numeric_cols = data.select_dtypes(exclude=[np.number]).columns
    if len(non_numeric_cols) > 0:
        print(f"Non-numeric columns found: {non_numeric_cols}")
        for col in non_numeric_cols:
            print(f"Unique values in {col}: {data[col].unique()}")
    
    return data

In [8]:
import numpy as np

def clean_data(data):
    # Check for non-numeric values in L, A, B columns
    for col in ["L", "A", "B"]:
        non_numeric_values = data[pd.to_numeric(data[col], errors='coerce').isnull()][col]
        if not non_numeric_values.empty:
            print(f"Non-numeric values found in {col}: {non_numeric_values.unique()}")
            data = data[pd.to_numeric(data[col], errors='coerce').notnull()]
    
    # Convert L, A, B columns to numeric
    data["L"] = pd.to_numeric(data["L"], errors='coerce')
    data["A"] = pd.to_numeric(data["A"], errors='coerce')
    data["B"] = pd.to_numeric(data["B"], errors='coerce')
    
    # Drop rows with NaN values in L, A, B columns
    data = data.dropna(subset=["L", "A", "B"])
    
    return data

In [9]:
def preprocess_data(data):
    # Extract input features (LAB values)
    X = data[["L", "A", "B"]].values.astype(np.float32)
    
    # Normalize L, A, B channels
    X[:, 0] /= 100.0  # Normalize L channel: [0, 100] → [0, 1]
    X[:, 1] = (X[:, 1] + 128) / 255.0  # Normalize A channel: [-128, 127] → [0, 1]
    X[:, 2] = (X[:, 2] + 128) / 255.0  # Normalize B channel: [-128, 127] → [0, 1]
    
    # Extract target labels (Hue, Value, Chroma)
    y_hue = pd.get_dummies(data["Hue"]).values.astype(np.float32)  # One-hot encode Hue
    y_value = data["Value"].values.astype(np.float32).reshape(-1, 1)
    y_chroma = data["Chroma"].values.astype(np.float32).reshape(-1, 1)
    
    return X, y_hue, y_value, y_chroma

In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model

def build_model(input_shape, num_hues):
    inputs = Input(shape=input_shape)
    
    x = Dense(128, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    hue_output = Dense(num_hues, activation='softmax', name="hue_output")(x)
    value_output = Dense(1, activation='linear', name="value_output")(x)
    chroma_output = Dense(1, activation='linear', name="chroma_output")(x)
    
    return Model(inputs=inputs, outputs=[hue_output, value_output, chroma_output])

# Implement simplified quantization without tfmot
def apply_quantization(model):
    # Using TensorFlow Lite's quantization directly
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    quantized_tflite_model = converter.convert()
    
    # Save the quantized model to a temporary file and reload
    with open('temp_quantized_model.tflite', 'wb') as f:
        f.write(quantized_tflite_model)
    
    # Note: Returning the original model since we can't directly convert TFLite back to Keras
    # The quantized model will be used during save_model
    print("Model quantized and saved as 'temp_quantized_model.tflite'")
    return model

# Implement pruning without tfmot
def apply_pruning(model):
    # Simple magnitude-based pruning implementation
    # Create a new model with the same architecture but apply masks to small weights
    pruned_model = tf.keras.models.clone_model(model)
    pruned_model.set_weights(model.get_weights())
    
    # Define sparsity level (percentage of weights to be zeroed)
    sparsity = 0.5  # 50% sparsity
    
    # Apply pruning to each layer with weights
    for layer in pruned_model.layers:
        if isinstance(layer, tf.keras.layers.Dense):
            weights = layer.get_weights()
            if len(weights) > 0:  # Check if layer has weights
                # Apply pruning to weights (not biases)
                w = weights[0]
                # Get threshold value for the specified sparsity
                threshold = tf.sort(tf.abs(tf.reshape(w, [-1])))[int(w.size * sparsity)]
                # Create a mask that zeros out weights below threshold
                mask = tf.cast(tf.abs(w) >= threshold, w.dtype)
                # Apply mask
                weights[0] = w * mask
                layer.set_weights(weights)
    
    print(f"Model pruned with {sparsity*100}% sparsity")
    return pruned_model

def save_model(model, file_name):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()
    
    with open(file_name, 'wb') as f:
        f.write(tflite_model)
    print(f"Model saved as {file_name}")



In [11]:
def main():
    # Step 1: Load the dataset
    file_path = "/home/sala/data/equivalent_munsell.csv"  # Replace with the actual path to your dataset
    print("Loading data...")
    data = load_data(file_path)
    
    # Step 2: Clean the dataset
    print("Cleaning data...")
    data = clean_data(data)
    
    # Step 3: Preprocess the dataset
    print("Preprocessing data...")
    X, y_hue, y_value, y_chroma = preprocess_data(data)
    
    # Split into training and validation sets
    split_idx = int(len(X) * 0.8)
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_hue_train, y_hue_val = y_hue[:split_idx], y_hue[split_idx:]
    y_value_train, y_value_val = y_value[:split_idx], y_value[split_idx:]
    y_chroma_train, y_chroma_val = y_chroma[:split_idx], y_chroma[split_idx:]
    
    # Step 4: Build the model
    num_hues = y_hue.shape[1]  # Number of unique hues (one-hot encoded)
    input_shape = (3,)  # LAB channels
    print("Building model...")
    model = build_model(input_shape, num_hues)
    
    # Compile the model
    model.compile(
        optimizer='adam',
        loss={
            "hue_output": "categorical_crossentropy",
            "value_output": "mse",
            "chroma_output": "mse"
        },
        metrics={
            "hue_output": "accuracy",
            "value_output": "mae",
            "chroma_output": "mae"
        }
    )
    
    # Step 5: Train the model
    print("Training model...")
    model.fit(
        X_train,
        {"hue_output": y_hue_train, "value_output": y_value_train, "chroma_output": y_chroma_train},
        validation_data=(X_val, {"hue_output": y_hue_val, "value_output": y_value_val, "chroma_output": y_chroma_val}),
        epochs=20,
        batch_size=32
    )
    
    # Step 6: Apply pruning
    print("Applying pruning...")
    pruned_model = apply_pruning(model)
    
    # Step 7: Apply quantization
    print("Applying quantization...")
    quantized_model = apply_quantization(pruned_model)
    
    # Step 8: Save the optimized model
    print("Saving optimized model...")
    save_model(quantized_model, "optimized_color_model.tflite")

if __name__ == "__main__":
    main()

Loading data...
   Hue  Value  Chroma         r         g         b         L         A  \
0  10B    1.0       1  0.069966  0.102321  0.125645  8.766975 -1.873666   
1  10B    1.0       2  0.047615  0.104012  0.144154  8.752576 -2.435589   
2  10B    1.0       3  0.012557  0.106032  0.164007  8.734874 -3.121113   
3  10B    1.0       4  0.000000  0.108210  0.184417  9.105021 -1.903252   
4  10B    1.0       5  0.000000  0.110270  0.204285  9.653357  0.194611   

           B  
0  -5.355077  
1  -8.942874  
2 -12.765529  
3 -16.018042  
4 -18.828227  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10447 entries, 0 to 10446
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Hue     10447 non-null  object 
 1   Value   10447 non-null  float64
 2   Chroma  10447 non-null  int64  
 3   r       10447 non-null  float64
 4   g       10447 non-null  float64
 5   b       10447 non-null  float64
 6   L       10447 non-null  float64
 7

I0000 00:00:1742781926.500903   88295 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Training model...
Epoch 1/20


I0000 00:00:1742781929.532176   90856 service.cc:152] XLA service 0x7fc9dc0030d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742781929.532367   90856 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2025-03-24 07:35:29.606032: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742781929.897846   90856 cuda_dnn.cc:529] Loaded cuDNN version 90300


 29/262 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - chroma_output_loss: 243.5101 - chroma_output_mae: 12.8940 - hue_output_accuracy: 0.0321 - hue_output_loss: 4.4414 - loss: 283.9442 - value_output_loss: 35.9927 - value_output_mae: 5.5189

...

INFO:tensorflow:Assets written to: /tmp/tmpp__a27hj/assets


Saved artifact at '/tmp/tmpp__a27hj'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  List[TensorSpec(shape=(None, 41), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)]
Captures:
  140509253851024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509253847568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509253846416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509253849872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509253850832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509253847760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509253850640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509253856976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509253859472: TensorSpec(shape=(), dty

W0000 00:00:1742781981.517095   88295 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742781981.517168   88295 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


Model saved as optimized_color_model.tflite


2025-03-24 07:36:21.517399: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpp__a27hj
2025-03-24 07:36:21.518121: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-24 07:36:21.518132: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpp__a27hj
2025-03-24 07:36:21.525362: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-24 07:36:21.561910: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpp__a27hj
2025-03-24 07:36:21.573493: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 56100 microseconds.


#evaluation

In [14]:
# Step 3: Load the trained model
def load_model(model_path):
    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

# Step 4: Run inference on the test data
def run_inference(interpreter, X_test):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    predictions = []
    for i in range(len(X_test)):
        # Prepare input data
        input_data = np.expand_dims(X_test[i], axis=0).astype(input_details[0]['dtype'])
        
        # Set input tensor
        interpreter.set_tensor(input_details[0]['index'], input_data)
        
        # Run inference
        interpreter.invoke()
        
        # Get output tensors
        hue_output = interpreter.get_tensor(output_details[0]['index'])
        value_output = interpreter.get_tensor(output_details[1]['index'])
        chroma_output = interpreter.get_tensor(output_details[2]['index'])
        
        # Append predictions
        predictions.append({
            "hue": np.argmax(hue_output),  # Predicted class for Hue
            "value": value_output[0][0],   # Predicted Value
            "chroma": chroma_output[0][0]  # Predicted Chroma
        })
    
    return predictions

# Step 5: Evaluate the model
def evaluate_model(predictions, y_hue_test, y_value_test, y_chroma_test):
    # Extract predictions
    pred_hue = np.array([p["hue"] for p in predictions])
    pred_value = np.array([p["value"] for p in predictions])
    pred_chroma = np.array([p["chroma"] for p in predictions])
    
    # Compute metrics
    hue_accuracy = accuracy_score(np.argmax(y_hue_test, axis=1), pred_hue)
    value_mae = mean_absolute_error(y_value_test, pred_value)
    chroma_mae = mean_absolute_error(y_chroma_test, pred_chroma)
    
    print(f"Hue Accuracy: {hue_accuracy * 100:.2f}%")
    print(f"Value MAE: {value_mae:.4f}")
    print(f"Chroma MAE: {chroma_mae:.4f}")

# Main function
def main():
    # Paths
    file_path = "/home/sala/data/equivalent_munsell.csv"
    model_path = "/home/sala/iit/dsgp/soil_analysis_automation_system/color_features/notebooks/optimized_color_model.tflite"  
    # Load and preprocess the dataset
    data = load_data(file_path)
    X, y_hue, y_value, y_chroma = preprocess_data(data)
    
    # Split into training and test sets
    split_idx = int(len(X) * 0.8)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_hue_train, y_hue_test = y_hue[:split_idx], y_hue[split_idx:]
    y_value_train, y_value_test = y_value[:split_idx], y_value[split_idx:]
    y_chroma_train, y_chroma_test = y_chroma[:split_idx], y_chroma[split_idx:]
    
    # Load the trained model
    interpreter = load_model(model_path)
    
    # Run inference
    predictions = run_inference(interpreter, X_test)
    
    # Evaluate the model
    evaluate_model(predictions, y_hue_test, y_value_test, y_chroma_test)

if __name__ == "__main__":
    main()

   Hue  Value  Chroma         r         g         b         L         A  \
0  10B    1.0       1  0.069966  0.102321  0.125645  8.766975 -1.873666   
1  10B    1.0       2  0.047615  0.104012  0.144154  8.752576 -2.435589   
2  10B    1.0       3  0.012557  0.106032  0.164007  8.734874 -3.121113   
3  10B    1.0       4  0.000000  0.108210  0.184417  9.105021 -1.903252   
4  10B    1.0       5  0.000000  0.110270  0.204285  9.653357  0.194611   

           B  
0  -5.355077  
1  -8.942874  
2 -12.765529  
3 -16.018042  
4 -18.828227  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10447 entries, 0 to 10446
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Hue     10447 non-null  object 
 1   Value   10447 non-null  float64
 2   Chroma  10447 non-null  int64  
 3   r       10447 non-null  float64
 4   g       10447 non-null  float64
 5   b       10447 non-null  float64
 6   L       10447 non-null  float64
 7   A       10447

/home/sala/miniconda3/lib/python3.12/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


NameError: name 'accuracy_score' is not defined

In [ ]:
# Define Different Model Architectures
def simple_cnn(input_shape=(224, 224, 3), num_classes=5):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return Model(inputs, outputs)



In [ ]:
# Experiment with Hyperparameters
def train_model(model, train_ds, val_ds, epochs=50, learning_rate=3e-4):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    checkpoint = ModelCheckpoint(
        'models/checkpoints/best_model.h5',
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1
    )
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
    
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=[checkpoint, early_stopping]
    )
    
    return history



In [15]:
# Load Data
train_ds = tf.keras.utils.image_dataset_from_directory(
    'data/soil/labeled',
    image_size=(224, 224),
    batch_size=32,
    label_mode='int',
    validation_split=0.2,
    subset='training',
    seed=42
).map(build_data_pipeline(), num_parallel_calls=tf.data.AUTOTUNE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'data/soil/labeled',
    image_size=(224, 224),
    batch_size=32,
    label_mode='int',
    validation_split=0.2,
    subset='validation',
    seed=42
).map(build_data_pipeline(), num_parallel_calls=tf.data.AUTOTUNE)

# Experiment 1: Simple CNN
simple_cnn_model = simple_cnn()
history_simple_cnn = train_model(simple_cnn_model, train_ds, val_ds, epochs=10)

# Experiment 2: MobileNetV3-LAB
mobile_net_model = mobile_soil_classifier(num_classes=5)
history_mobile_net = train_model(mobile_net_model, train_ds, val_ds, epochs=10)

# Plot Training History
def plot_training_history(history, model_name):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

plot_training_history(history_simple_cnn, 'Simple CNN')
plot_training_history(history_mobile_net, 'MobileNetV3-LAB')

# Try Different Training Strategies
# Experiment 3: MobileNetV3-LAB with Quantization and Pruning
optimized_model = optimize_for_mobile(mobile_net_model)
history_optimized = train_model(optimized_model, train_ds, val_ds, epochs=10)

# Evaluate Model Performance
def evaluate_model(model, val_ds):
    loss, accuracy = model.evaluate(val_ds)
    print(f'Validation Loss: {loss}')
    print(f'Validation Accuracy: {accuracy}')

evaluate_model(optimized_model, val_ds)

NotFoundError: Could not find directory data/soil/labeled

In [16]:
# Try Different Training Strategies
# Experiment 3: MobileNetV3-LAB with Quantization and Pruning
optimized_model = optimize_for_mobile(mobile_net_model)
history_optimized = train_model(optimized_model, train_ds, val_ds, epochs=10)

# Evaluate Model Performance
def evaluate_model(model, val_ds):
    loss, accuracy = model.evaluate(val_ds)
    print(f'Validation Loss: {loss}')
    print(f'Validation Accuracy: {accuracy}')

evaluate_model(optimized_model, val_ds)

NameError: name 'optimize_for_mobile' is not defined